# zombie detection

<img src='https://drive.google.com/uc?export=view&id=18Ck0qNSZy9F1KsUKWc4Jv7_x_1e_fXTN' alt='zombie'>

## installation

In [ ]:
!rm -rf ./models/
!git clone --depth 1 https://github.com/tensorflow/models/

In [ ]:
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%writefile models/research/setup.py

import os
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    'tf-models-official==2.8.0',
    'tensorflow_io'
]

setup(
    name='object_detection',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=(
        [p for p in find_packages() if p.startswith('object_detection')] +
        find_packages(where=os.path.join('.', 'slim'))),
    package_dir={
        'datasets': os.path.join('slim', 'datasets'),
        'nets': os.path.join('slim', 'nets'),
        'preprocessing': os.path.join('slim', 'preprocessing'),
        'deployment': os.path.join('slim', 'deployment'),
        'scripts': os.path.join('slim', 'scripts'),
    },
    description='Tensorflow Object Detection Library',
    python_requires='>3.6',
)

In [ ]:
!python -m pip install models/research

## imports

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import zipfile
import io
import scipy.misc
import numpy as np

import glob
import imageio
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:
import tensorflow as tf
from PIL import Image
from object_detection.utils import label_map_util, config_util, colab_utils, visualization_utils as viz_utils
from object_detection.builders import model_builder

## utilities

In [ ]:
def load_image_into_numpy_array(path):    
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
    image_np_with_annotations = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.8)
    
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    
    else:
        plt.imshow(image_np_with_annotations)


## download the zombie data

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training-zombie.zip \
    -O ./training-zombie.zip

local_zip = './training-zombie.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./training')
zip_ref.close()

### visualize the training images

In [ ]:
%matplotlib inline
train_image_dir = 'training'
train_images_np = []
for i in range(1, 6):
    image_path = os.path.join(f"{os.getcwd()}/{train_image_dir}/training-zombie{i}.jpg")
    image = Image.open(image_path)
    train_images_np.append(np.array(image))

plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.labelsize'] = False
plt.rcParams['ytick.labelsize'] = False
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['figure.figsize'] = [14, 7]

for idx, train_image_np in enumerate(train_images_np):
    plt.subplot(1, 5, idx+1)
    plt.imshow(train_image_np)

plt.show()

## prepare data for training

In [ ]:
ref_gt_boxes = [
        np.array([[0.27333333, 0.41500586, 0.74333333, 0.57678781]]),
        np.array([[0.29833333, 0.45955451, 0.75666667, 0.61078546]]),
        np.array([[0.40833333, 0.18288394, 0.945, 0.34818288]]),
        np.array([[0.16166667, 0.61899179, 0.8, 0.91910903]]),
        np.array([[0.28833333, 0.12543962, 0.835, 0.35052755]]),
      ]

gt_boxes = ref_gt_boxes
for gt_box in gt_boxes:
    try:
      assert(gt_box is not None)
    
    except:
      gt_boxes = ref_gt_boxes
      break

#### view ground truth box coordinates

In [ ]:
for gt_box in gt_boxes:
  print(gt_box)

### define the category index dictionary


In [ ]:
zombie_class_id = 1
category_index = {zombie_class_id: {'id': zombie_class_id, 'name': 'zombie'}}
num_classes = 1

In [ ]:
print(category_index[zombie_class_id])

### data preprocessing

In [ ]:
label_id_offset = 1
train_image_tensors = []
gt_classes_one_hot_tensors = []
gt_box_tensors = []

for (train_image_np, gt_box_np) in zip(train_images_np, gt_boxes):
    train_image_tensors.append(tf.expand_dims(tf.convert_to_tensor(
        train_image_np, dtype=tf.float32), axis=0))
    
    gt_box_tensors.append(tf.convert_to_tensor(gt_box_np, dtype=tf.float32))
    
    zero_indexed_groundtruth_classes = tf.convert_to_tensor(
        np.ones(shape=[gt_box_np.shape[0]], dtype=np.int32) - label_id_offset)
    
    gt_classes_one_hot_tensors.append(tf.one_hot(
        zero_indexed_groundtruth_classes, num_classes))

print('Done prepping data.')

## visualize the zombies with their ground truth bounding boxes


In [ ]:
dummy_scores = np.array([1.0], dtype=np.float32)
plt.figure(figsize=(30, 15))
for idx in range(5):
    plt.subplot(2, 4, idx+1)
    plot_detections(
      train_images_np[idx],
      gt_boxes[idx],
      np.ones(shape=[gt_boxes[idx].shape[0]], dtype=np.int32),
      dummy_scores, category_index)

plt.show()

## download the checkpoint containing the pre-trained weights

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

## configure the model

In [ ]:
tf.keras.backend.clear_session()
pipeline_config = "/content/models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config"
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
configs

In [ ]:
model_config = configs['model']
model_config

### modify model config

In [ ]:
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
model_config

## build the model

In [ ]:
detection_model = model_builder.build(model_config, is_training=True)
print(type(detection_model))

## restore weights from checkpoint

In [ ]:
detection_model

#### find the source code for detection_model

In [ ]:
vars(detection_model)

In [ ]:
detection_model._box_predictor

In [ ]:
vars(detection_model._box_predictor)

### define checkpoints for the box predictor

In [ ]:
tmp_box_predictor_checkpoint = tf.train.Checkpoint(_box_predictor=detection_model._box_predictor)

In [ ]:
type(tmp_box_predictor_checkpoint)

In [ ]:
vars(tmp_box_predictor_checkpoint)

### define the temporary model checkpoint


In [ ]:
tmp_model_checkpoint = tf.train.Checkpoint(_box_predictor=tmp_box_predictor_checkpoint, _feature_extractor=detection_model._feature_extractor)

In [ ]:
type(tmp_model_checkpoint)

In [ ]:
vars(tmp_model_checkpoint)

### restore the checkpoint

In [ ]:
checkpoint_path = "/content/models/research/object_detection/test_data/checkpoint/ckpt-0"
checkpoint = tf.train.Checkpoint(model=tmp_model_checkpoint)
checkpoint.restore(checkpoint_path)

### run a dummy image to generate the model variables

In [ ]:
dummy = tf.zeros((4, 640, 640, 3))

In [ ]:
tmp_image, tmp_shapes = detection_model.preprocess(dummy)
tmp_prediction_dict = detection_model.predict(tmp_image, tmp_shapes)
tmp_detections = detection_model.postprocess(tmp_prediction_dict, tmp_shapes)
print('Weights restored!')

In [ ]:
assert len(detection_model.trainable_variables) > 0, "Please pass in a dummy image to create the trainable variables."
print(detection_model.weights[0].shape)
print(detection_model.weights[231].shape)
print(detection_model.weights[462].shape)

### set training hyperparameters

In [ ]:
tf.keras.backend.set_learning_phase(True)
batch_size = 4
num_batches = 100
learning_rate = .01
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

## choose layers to fine-tune

In [ ]:
for i,v in enumerate(detection_model.trainable_variables):
    print(f"i: {i} \t name: {v.name} \t shape:{v.shape} \t dtype={v.dtype}")

### select prediction layer variables

In [ ]:
to_fine_tune = []
for v in detection_model.trainable_variables:
  if v.name.startswith('WeightSharedConvolutionalBoxPredictor'):
    to_fine_tune.append(v)

In [ ]:
print(to_fine_tune[0].name)
print(to_fine_tune[2].name)

## train model

In [ ]:
g_images_list = train_image_tensors[0:2]

In [ ]:
g_preprocessed_image = detection_model.preprocess(g_images_list[0])
print(f"g_preprocessed_image type: {type(g_preprocessed_image)}")
print(f"g_preprocessed_image length: {len(g_preprocessed_image)}")
print(f"index 0 has the preprocessed image of shape {g_preprocessed_image[0].shape}")
print(f"index 1 has information about the image's true shape excluding padding: {g_preprocessed_image[1]}")

In [ ]:
preprocessed_image_list = []
true_shape_list = []

for img in g_images_list:
    processed_img, true_shape = detection_model.preprocess(img)
    preprocessed_image_list.append(processed_img)
    true_shape_list.append(true_shape)

print(f"preprocessed_image_list is of type {type(preprocessed_image_list)}")
print(f"preprocessed_image_list has length {len(preprocessed_image_list)}")
print()
print(f"true_shape_list is of type {type(true_shape_list)}")
print(f"true_shape_list has length {len(true_shape_list)}")

## make prediction

In [ ]:
# Try to call `predict` and pass in lists; look at the error message
try:
    detection_model.predict(preprocessed_image_list, true_shape_list)
except AttributeError as e:
    print("Error message:", e)

In [ ]:
preprocessed_image_tensor = tf.concat(preprocessed_image_list, axis=0)
true_shape_tensor = tf.concat(true_shape_list, axis=0)

print(f"preprocessed_image_tensor shape: {preprocessed_image_tensor.shape}")
print(f"true_shape_tensor shape: {true_shape_tensor.shape}")

In [ ]:
prediction_dict = detection_model.predict(preprocessed_image_tensor, true_shape_tensor)
print("keys in prediction_dict:")
for key in prediction_dict.keys():
    print(key)

#### calculate loss

In [ ]:
try:
    losses_dict = detection_model.loss(prediction_dict, true_shape_tensor)
except RuntimeError as e:
    print(e)

In [ ]:
gt_boxes_list = gt_box_tensors[0:2]
gt_classes_list = gt_classes_one_hot_tensors[0:2]
detection_model.provide_groundtruth(groundtruth_boxes_list=gt_boxes_list, groundtruth_classes_list=gt_classes_list)

In [ ]:
losses_dict = detection_model.loss(prediction_dict, true_shape_tensor)
print(f"loss dictionary keys: {losses_dict.keys()}")
print(f"localization loss {losses_dict['Loss/localization_loss']:.8f}")
print(f"classification loss {losses_dict['Loss/classification_loss']:.8f}")

In [ ]:
detection_model.provide_groundtruth(groundtruth_boxes_list=[], groundtruth_classes_list=[])

## define training step

In [ ]:
@tf.function
def train_step_fn(image_list,
                groundtruth_boxes_list,
                groundtruth_classes_list,
                model,
                optimizer,
                vars_to_fine_tune):
    with tf.GradientTape() as tape:
        preprocessed_image_list, true_shape_list = [], []
        for img in image_list:
          processed_img, true_shape = model.preprocess(img)
          preprocessed_image_list.append(processed_img), true_shape_list.append(true_shape)
        preprocessed_image_tensor = tf.concat(preprocessed_image_list, axis=0)
        true_shape_tensor = tf.concat(true_shape_list, axis=0)
        model.provide_groundtruth(groundtruth_boxes_list=groundtruth_boxes_list, groundtruth_classes_list=groundtruth_classes_list)
        prediction_dict = model.predict(preprocessed_image_tensor, true_shape_tensor)
        losses_dict = model.loss(prediction_dict, true_shape_tensor)
        total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
        gradients = tape.gradient([total_loss], to_fine_tune)
        optimizer.apply_gradients(zip(gradients, to_fine_tune))
    return total_loss

## training loop

In [ ]:
print('Start fine-tuning!', flush=True)
for idx in range(num_batches):
    all_keys = list(range(len(train_images_np)))
    random.shuffle(all_keys)
    example_keys = all_keys[:batch_size]
    gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
    gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
    image_tensors = [train_image_tensors[key] for key in example_keys]
    total_loss = train_step_fn(image_tensors, 
                               gt_boxes_list, 
                               gt_classes_list,
                               detection_model,
                               optimizer,
                               to_fine_tune
                              )

    if idx % 10 == 0:
        print('batch ' + str(idx) + ' of ' + str(num_batches)
        + ', loss=' +  str(total_loss.numpy()), flush=True)
print('Done fine-tuning!')

### Load test images and run inference with new model

In [ ]:
!rm zombie-walk-frames.zip
!rm -rf ./zombie-walk
!rm -rf ./results

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/zombie-walk-frames.zip \
    -O zombie-walk-frames.zip

local_zip = './zombie-walk-frames.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./results')
zip_ref.close()

### load images into numpy arrays to prepare it for inference.

In [ ]:
test_image_dir = './results/'
test_images_np = []

for i in range(0, 237):
    image_path = os.path.join(test_image_dir, 'zombie-walk' + "{0:04}".format(i) + '.jpg')
    print(image_path)
    test_images_np.append(np.expand_dims(
      load_image_into_numpy_array(image_path), axis=0))

### Preprocess, predict, and post process an image

In [ ]:
@tf.function
def detect(input_tensor):
    preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    prediction_dict = detection_model.predict(preprocessed_image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)    
    return detections

In [ ]:
label_id_offset = 1
results = {'boxes': [], 'scores': []}

for i in range(len(test_images_np)):
    input_tensor = tf.convert_to_tensor(test_images_np[i], dtype=tf.float32)
    detections = detect(input_tensor)
    plot_detections(
      test_images_np[i][0],
      detections['detection_boxes'][0].numpy(),
      detections['detection_classes'][0].numpy().astype(np.uint32)
      + label_id_offset,
      detections['detection_scores'][0].numpy(),
      category_index, figsize=(15, 20), image_name="./results/gif_frame_" + ('%03d' % i) + ".jpg")
    results['boxes'].append(detections['detection_boxes'][0][0].numpy())
    results['scores'].append(detections['detection_scores'][0][0].numpy())

In [ ]:
print(len(results['boxes']))
print(results['boxes'][0].shape)
print()

print(np.allclose(results['boxes'][0], [0.28838485, 0.06830047, 0.7213766 , 0.19833465], rtol=0.18))
print(np.allclose(results['boxes'][5], [0.29168868, 0.07529271, 0.72504973, 0.20099735], rtol=0.18))
print(np.allclose(results['boxes'][10], [0.29548776, 0.07994056, 0.7238164 , 0.20778716], rtol=0.18))

In [ ]:
x = np.array(results['scores'])

zombie_detected = (np.where(x > 0.9, 1, 0).sum())/237*100
print(zombie_detected)

In [ ]:
print('Frame 0')
display(IPyImage('./results/gif_frame_000.jpg'))
print()
print('Frame 5')
display(IPyImage('./results/gif_frame_005.jpg'))
print()
print('Frame 10')
display(IPyImage('./results/gif_frame_010.jpg'))

## Create a zip of the zombie-walk images. 
You can download this if you like to create your own animations

In [ ]:
zipf = zipfile.ZipFile('./zombie.zip', 'w', zipfile.ZIP_DEFLATED)

filenames = glob.glob('./results/gif_frame_*.jpg')
filenames = sorted(filenames)

for filename in filenames:
    zipf.write(filename)

zipf.close()

## Create Zombie animation

In [ ]:
imageio.plugins.freeimage.download()

!rm -rf ./results/zombie-anim.gif

anim_file = './zombie-anim.gif'

filenames = glob.glob('./results/gif_frame_*.jpg')
filenames = sorted(filenames)
last = -1
images = []

for filename in filenames:
    image = imageio.imread(filename)
    images.append(image)

imageio.mimsave(anim_file, images, 'GIF-FI', fps=10)

Unfortunately, using `IPyImage` in the notebook (as you've done in the rubber ducky detection tutorial) for the large `gif` generated will disconnect the runtime. To view the animation, you can instead use the `Files` pane on the left and double-click on `zombie-anim.gif`. That will open a preview page on the right. It will take 2 to 3 minutes to load and see the walking zombie.

## Save results file for grading

Run the cell below to save your results. Download the `results.data` file and upload it to the grader in the classroom.

In [ ]:
import pickle

# remove file if it exists
!rm results.data

# write results to binary file. upload for grading.
with open('results.data', 'wb') as filehandle:
    pickle.dump(results['boxes'], filehandle)

print('Done saving! Please download `results.data` from the Files tab\n' \
      'on the left and submit for grading.\nYou can also use the next cell as a shortcut for downloading.')

In [ ]:
from google.colab import files

files.download('results.data')

**Congratulations on completing this assignment! Please go back to the Coursera classroom and upload `results.data` to the Graded Lab item for Week 2.**